# Project 1: Ranked-choice voting [40 marks]

In this project, you will write Python functions to tally election results from a set of ballots submitted by voters, using different vote counting systems.

You will write all your code as function definitions, in one of 4 modules: `ballot_data.py`, `pos.py`, `irv.py`, and `stv.py`.

- Each module should **only contain function definitions, and any required `import` statements** (for example if you need to import `numpy` to use in your function definitions).
- For each question, a few simple **automatic tests** will be provided in the notebook. You can use these to start checking that your function works as expected. They work similarly to your Coderunner quizzes.
    - In some cases, the tests use `assert` statements (or similar functions from [`np.testing`](https://numpy.org/doc/stable/reference/routines.testing.html)).
        - `assert X` will do nothing if `X` is `True`.
        - `assert X` will produce an `AssertionError` if `X` is `False`.
    - Some of the tests use helper functions to keep the notebook tidy, which you can see in `notebook_tests.py`.
    - The provided tests are minimal (similar to the "Examples" provided for pre-checking in the Coderunner quizzes). To make sure that your function works fully with all possible inputs, you need to test it yourself more extensively, using tests which consider different scenarios and possible edge cases.
    - Some of the tests load a small amount of ballot data from a `.txt` file. You can look at the data itself by looking at the content of the files.
    - To mark your assignment after you've submitted it, more automatic tests will be performed which will test your functions further (similar to the further tests in the Coderunner quizzes, which are run when you click "Check"). You will not see the results of these tests until the grades and feedback are returned to you. This means, in particular, that *passing all the tests provided in this notebook does not guarantee full marks.*
- The tasks in the assignment will ask you to write specific functions.
    - Use the function name, input arguments, and output values specified in the question.
    - You may wish to write extra functions to break down your code into smaller parts, so that the main function in each task calls your smaller functions instead of having all the code in a single function definition. This may be useful if your functions become too long, or if you find yourself re-using the same piece of code multiple times.
    - If you want to do so, you are allowed to add extra input arguments to the required functions. However, if you do, make sure to give them a default value, so that calling the function without specifying the extra input arguments still gives the expected result as described in the question. In other words, your code should still work exactly as described in the questions, without the need to provide extra information to your functions.
- All the functions which manipulate NumPy arrays can be written using loops. However, in some cases, it will be possible to write more efficient functions by making use of vectorised operations (i.e. doing computations on an entire row, column, or whole array all at once, without going over every element of the array one by one).
    - For full marks, your functions should avoid the use of loops if a more efficient implementation is possible with Numpy array operations.
    - Partial marks can still be achieved even if you use loops for everything. If you are struggling with writing a function, it may be easier to start with using loops, make sure your function gives the correct result, and only later think about whether you can make your code more efficient.
    - You will likely find many useful functions in the [NumPy documentation](https://numpy.org/doc/stable/reference/routines.html) and the [NumPy user guide](https://numpy.org/doc/stable/user/absolute_beginners.html) to manipulate or do computations on NumPy arrays in an efficient way.
- **Up to half** of the marks for a given question may be deducted for poor readability; missing, incomplete, or inaccurate code comments and/or docstrings; poor code structure and efficiency; or poorly displayed results. You can use the marking scheme for the code reviews for reference to make sure you submit good-quality code. Note that you are **not** required to check input argument values in this assignment.

In [ ]:
# Run this cell to import modules
import ballot_data as bd, pos, irv, stv
import numpy as np
import notebook_tests as nbt
import importlib

---

## Ranked-choice voting

To experiment with different vote counting systems, we first need some ballots to count. In all cases for this project, a number of voters have been asked to fill their ballots by **ranking their favourite candidates in order of preference**; this is called _ranked-choice voting_. Ranked choice voting is used in Scotland, to elect local council representatives.

For example, with 5 candidates, one filled ballot might look like this:

<div>
<img src="img/ballot.png" width="250px" alt="Filled ballot example"/>
</div>

This voter has selected Candidate 3 as their favourite candidate, and ranked them as first preference. They then ranked Candidate 5 as their second preference, and Candidate 2 as their third preference. They chose not to rank any other candidate -- presumably they really dislike Candidates 1 and 4.

All ballots must have **at least one candidate ranked** as first preference to be valid, but each voter can choose to rank anywhere between 1 and all candidates, in order of preference. The voter in the example above chose to rank 3 candidates.

### Counting methods

Different vote counting methods can be used to process the ballot data in different ways, which will lead to different results -- this depends largely on how second, third, etc. preferences are taken into account. This project will explore two different methods to calculate a winner based on ranked-choice ballots:

- **Positional voting**, where the rank given to each candidate on the ballot is worth a certain weight, or a certain number of points (this is equivalent). Normally, higher-ranked preference votes are given more weight than lower-ranked preferences.
- **Instant runoff voting**, where the candidates with the fewest votes are eliminated one-by-one. At each elimination, ballots which placed the eliminated candidate first are transferred to the candidates that were ranked as second preferences. This allows voters to vote for their favourite candidate, and to rank less preferred candidates in second, third, etc. preferences so that their vote doesn't go to waste if their favourite doesn't win.

### Ballot data representation

We choose to represent the ballot data using a rectangular NumPy array of integers, where:

- each **row** represents **one ballot** submitted by one voter;
- each **non-zero value** represents a **candidate**;
- each **column** represents **a preference rank**, from first to last preference going left to right, for example a `2` in the first column means that Candidate 2 was the voter's first preference;
- a value of `0` indicates no ranking.

The array will always have as many rows as there are valid ballots (or voters), and as many columns as there are candidates running in the election.

For example, all the ballots for an election with 5 candidates and 10 voters might look like this:

```
[[1 0 0 0 0]
 [4 2 1 0 0]
 [2 4 5 0 0]
 [3 2 1 4 5]
 [5 3 1 0 0]
 [3 4 0 0 0]
 [4 3 2 0 0]
 [3 4 0 0 0]
 [5 4 3 1 2]
 [3 5 2 0 0]]
```

where, for example:

- row 0 showing `[1 0 0 0 0]` means that the first voter has ranked Candidate 1 as their first preference, and chosen not to rank any other candidates;
- row 1 showing `[4 2 1 0 0]` means that the second voter has ranked Candidate 4 as their first preference, Candidate 2 as their second preference, and Candidate 1 as their third preference, but did not indicate a fourth or fifth preference;
- the last row showing `[3 5 2 0 0]` is the ballot shown in the picture above;
- etc.

The function `generate_ballots()` is provided in `ballot_data.py` to generate randomised ballots for a given number of voters and candidates, and a target distribution of probabilities for each candidate to be ranked as a particular preference. You should not modify it -- it's provided for your information, and you should feel free to use it to generate randomised datasets for testing your functions throughout the assignment.

---

## Counting the votes

Before we start calculating election results, we need to be able to count the total votes received by a given candidate, at any given preference rank.

In `ballot_data.py`, you are given a helper function `ballot_selector()` which takes 3 input arguments:

- `ballots`, a NumPy array containing ballot data as explained above, with one row per ballot and one column per preference rank,
- `candidate`, an integer between 1 and the total number of candidates, corresponding to a particular candidate,
- `preference`, an integer between 0 and the total number of candidates minus 1, with default value 0,

and returns a NumPy vector with the same length as the number of rows in `ballots` (the number of voters), containing `True` where the corresponding ballots have ranked candidate `candidate` as their preference at rank `preference`, and `False` everywhere else.

Essentially, this function returns a mask or a stencil which will allow us to select only certain ballots we're interested in. For an example of this, review Exercise 3 in the "NumPy arrays" notebook in your course materials (section 13); we can select certain parts of a NumPy array depending on a given condition.

For example, `selector = ballot_selector(ballots, candidate=4, preference=1)` returns a NumPy vector, whose `True` values correspond to all the rows of `ballots` where candidate 4 was ranked as second preference, i.e. where there is a `4` in the second column (indexed as column `1`). Here is this selection applied to the ballots in our example -- the **important feature** is that **candidate 4 is ranked as second preference** in all ballots selected using `selector`. (The selected ballots are indicated with `->` for clarity.)

```
     ballots       selector      ballots[selector]
   
   [[1 0 0 0 0]     [False         [[2 4 5 0 0] 
    [4 2 1 0 0]      False          [3 4 0 0 0] 
->  [2 4 5 0 0]      True           [3 4 0 0 0] 
    [3 2 1 4 5]      False          [5 4 3 1 2]]
    [5 3 1 0 0]      False 
->  [3 4 0 0 0]      True 
    [4 3 2 0 0]      False
->  [3 4 0 0 0]      True
->  [5 4 3 1 2]      True
    [3 5 2 0 0]]     False]
```

In [ ]:
# Run this cell to test the provided functions (don't change this code!)
importlib.reload(bd);                                  # Refresh any changes made to ballot_data.py
ballots = np.loadtxt('testing/tally.txt', dtype=int)   # Load ballot data exactly as in the example above
selector = bd.ballot_selector(ballots, candidate=4, preference=1)   # Use the function to select the ballots

# This should correspond to the above
print(selector)
print(ballots[selector])

Now that we have a way to select specific ballots, we can use this to count votes received by all candidates at any preference rank that we choose. We will need a function that counts votes and ranks candidates according to how many votes they obtained.

---

#### 🚩 Task 1 [4 marks]

In `ballot_data.py`, write a function `count_votes()` which takes 3 input arguments:

- `ballots`, a NumPy array containing ballot data, with one row per ballot and one column per candidate,
- `preference`, an integer between 0 and the total number of candidates minus 1, with default value `0`,
- `sort_by`, a string with default value `'count'`,

and returns 2 Numpy vectors (or lists) of the same length:

- `candidates`, a NumPy vector or list of the same length as the number of columns in `ballots`, containing all candidates in the election,
- `vote_counts`, a NumPy vector or list containing the total number of votes at the given `preference` for each candidate at the corresponding position in `candidates`.

If `sort_by='count'` (the default), the candidates and vote counts should be sorted by increasing number of votes. If `sort_by='candidate'`, the candidates and vote counts should be sorted by candidate number, e.g. `candidates` will be `[1 2 3 4 5]` if there are 5 candidates.

For example, counting the second preference votes using the same example as above, we have:

- Candidate 1 received the fewest second preference votes (zero).
- Candidate 5 is next, with 1 second preference vote.
- Candidates 2 and 3 are next, with 2 second preference votes each.
- Candidate 4 received the most second preference votes (4).

Therefore:

- `count_votes(ballots, preference=1)` should give `[1 5 2 3 4]` for `candidates`, and `[0 1 2 2 4]` for `vote_counts`.
- `count_votes(ballots, preference=1, sort_by='candidate')` should give `[1 2 3 4 5]` for `candidates`, and `[0 2 2 4 1]` for `vote_counts`.

With the default sorting by `'count'`, when there are tied candidates with the same number of votes, you can return them in any order in `candidates`; in our example, returning `[1 5 3 2 4]` would also be fine, since candidates 2 and 3 got the same number of second-preference votes.

In [ ]:
# Run this cell to test your function (don't change this code!)
importlib.reload(nbt);
importlib.reload(bd);
ballots = np.loadtxt('testing/tally.txt', dtype=int)

# Number of second preferences
candidates, vote_counts = bd.count_votes(ballots, preference=1)
nbt.assert_array_in(candidates, [[1, 5, 3, 2, 4], [1, 5, 2, 3, 4]])
nbt.assert_array_equal(vote_counts, [0, 1, 2, 2, 4])

# Number of third preferences, sorting by candidate
candidates, vote_counts = bd.count_votes(ballots, preference=2, sort_by='candidate')
nbt.assert_array_equal(candidates, [1, 2, 3, 4, 5])
nbt.assert_array_equal(vote_counts, [3, 2, 1, 0, 1])

print('Passed the tests.')

---

## Electing a candidate

Now that we have a way to count votes for all candidates at any level of preference, we can proceed to electing a candidate. There exist many different ways to count votes and elect a candidate from ranked-choice ballots; we will implement some of these.

---

***Note:*** when writing functions for the rest of the assignment, in `pos.py` and in `irv.py`, you will need to use your functions `ballot_selector()` and `count_votes()` from `ballot_data.py`.

You will see `import ballot_data as bd` at the top of `pos.py` and `irv.py`. **Don't change it**! With this statement, use `bd.ballot_selector()` and `bd.count_votes()` whenever you need to use any of the functions from `ballot_data.py`, as is done in the notebook.

---

## Positional voting

In **positional voting**, preferences are used to give different weights or points to the chosen candidates in each ballot. Here is an example: with 3 candidates, we could decide that

- first preferences are worth 3 points,
- second preferences are worth 2 points,
- third preferences are worth 1 point.

This means that a first-preference vote is worth 3 times the weight of a third-preference vote, for instance.

The votes are counted by counting all the first, second, etc. preferences for each candidate, each time multiplying the count by the corresponding weights, and summing the total. For instance, if there are 7 ballots with ranked preferences for our 3 candidates:

```
[[2 0 0]
 [1 2 3]
 [1 2 0]
 [1 2 3]
 [2 3 1]
 [2 0 0]
 [1 2 0]]
```

Counting the number of first, second, and third preferences for each candidate, we have:

| | Candidate 1 | Candidate 2 | Candidate 3 |
|:--:|:--:|:--:|:--:|
| First preferences (3 pts) | 4 | 3 | 0 |
| Second preferences (2 pts) | 0 | 4 | 1 |
| Third preferences (1 pt) | 1 | 0 | 2 |

Finally, counting the points:
- Candidate 1 gets $(4\times 3) + (0 \times 2) + (1 \times 1) = 13$ points.
- Candidate 2 gets $(3\times 3) + (4 \times 2) + (0 \times 1) = 17$ points.
- Candidate 3 gets $(0\times 3) + (1 \times 2) + (2 \times 1) = 4$ points.

Candidate 2 is elected with 17 points.

The weights in this example are `[3, 2, 1]`, but this is not the only possible choice. For instance, some systems would use `[2, 1, 0]`; others use `[1, 1/2, 1/3]`; etc.

---

#### 🚩 Task 2 [3 marks]

In `pos.py`, write a function `positional_voting()` which takes 2 input arguments:

- `ballots`, a NumPy array containing ballot data as before,
- `weights`, a list with the same length as the number of candidates, containing the weights attributed to each preference in order (i.e. first preference weight, then second preference, third preference, etc. from left to right).

Your function should calculate the total number of points obtained by each candidate using the positional voting system with the weights given in `weights`, and return 2 output values:

- `candidates`, a NumPy vector or list containing all candidates in the election, sorted by increasing total number of points,
- `points`, a NumPy vector or list containing the total number of points obtained by each candidate, sorted by increasing total number of points.

Note that weights (and therefore point totals) will not necessarily always be integers.

For instance, using the example above, `positional_voting(ballots, [3, 2, 1])` should return `[3 1 2]` for `candidates` and `[4 13 17]` for `points`; the winner of the election is candidate 2 with 17 points.

In [ ]:
importlib.reload(nbt);
importlib.reload(pos);
ballots = np.loadtxt('testing/positional.txt', dtype=int)

# Testing the proposed example
candidates, points = pos.positional_voting(ballots, [3, 2, 1])
nbt.assert_array_equal(candidates, [3, 1, 2])
nbt.assert_array_equal(points, [4, 13, 17])

# Choosing weights to only count 3rd preferences
candidates, points = pos.positional_voting(ballots, [0, 0, 1])
nbt.assert_array_equal(candidates, [2, 1, 3])
nbt.assert_array_equal(points, [0, 1, 2])

print('Passed the tests.')

---

## Instant runoff vote

Another way to elect a candidate using ranked-choice ballots is called **instant-runoff vote**. To win, a candidate needs to be the last candidate standing after all others are eliminated.

Here is how we proceed:

1. We start by counting the **first preference** votes. This is our baseline number of votes for each candidate.
2. We then find the **least popular candidate**, looking at the number of first preference votes; we **eliminate** this candidate.
3. We look at all the ballots which had the eliminated candidate as first preference, and **redistribute** the votes according to the **second preferences** on those ballots.
4. After eliminating one candidate and redistributing their votes to the others, we go back to step 1 until we have eliminated all candidates except one. They are the winner.

We'll break this process down into several parts.

---

### Redistributing the votes

When a candidate is eliminated, we will need to redistribute votes to second preferences; we can actually do this by updating our `ballots` array. We will start by implementing this functionality, as it should make the next functions a bit easier to think about.

Here is an example, with 3 ballots and 5 candidates. Candidate 3 was not ranked at all by the first voter, they were third preference for the second voter, and they were first preference for the third voter. If Candidate 3 is eliminated, we want to update our ballots as follows:

```
   before          after
[1 4 0 0 0]     [1 4 0 0 0]
[1 5 3 2 4]     [1 5 2 4 0]
[3 5 4 1 0]     [5 4 1 0 0]
```

The effects of this are:

- Candidate 3 is now completely removed in all ballots -- they are effectively eliminated from the competition.
- The first-preference vote in the third ballot for Candidate 3 (now eliminated) is now going to Candidate 5 (which was the voter's second preference) -- this is exactly what we needed.
- The third-preference vote of the second ballot is now going to Candidate 2, which was the next preferred candidate indicated by the voter after Candidate 3.
- Preferences are still given consecutively, without gaps -- this makes it easier to deal with future eliminations. The updated `ballots` array has the same format as the original array, but with one fewer candidate in the race.

---

#### 🚩 Task 3 [4 marks]

In `irv.py`, write a function `update_ballots()` which takes 2 input arguments:

- `ballots`, a NumPy array containing ballot data as before,
- `to_eliminate`, an integer between 1 and the total number of candidates, which corresponds to a candidate to eliminate,

and returns `updated_ballots`, a modified **copy** of the original `ballots` array, where the relevant ballots have been updated as shown above. Make sure to copy the array and return the modified copy; do not modify the original array.

In [ ]:
importlib.reload(irv);
importlib.reload(nbt);
ballots = np.loadtxt('testing/previous_ballots.txt', dtype=int)
expected = np.loadtxt('testing/updated_ballots.txt', dtype=int)
updated_ballots = irv.update_ballots(ballots, 3)
nbt.assert_array_equal(updated_ballots, expected)
print('Passed test.')

---

#### 🚩 Task 4 [3 marks]

We now wish to have an overview of how many votes are transferred to each remaining candidate after an elimination. We can do so by comparing what the ballots look like before and after an elimination.

In `irv.py`, write a function `vote_transfers()` which takes 2 input arguments:

- `previous_ballots` and `updated_ballots`, both NumPy arrays containing ballot data as before, where `updated_ballots` could for example represent `previous_ballots` modified by the `irv.update_ballots()` function after a candidate has been eliminated,

and returns 3 output values:

- `candidates`, a NumPy vector or list containing all candidates in the election in numerical order (e.g. `[1 2 3 ...]`),
- `vote_diffs`, a NumPy vector or list of the same length as `candidates`, containing the differences in first-preference votes obtained by each of the candidates between `previous_ballots` and `updated_ballots` (given in the same order as `candidates`),
- `discarded_diff`, a non-negative integer indicating how many ballots were discarded between `previous_ballots` and `updated_ballots`. For example, a ballot `[3 0 0 0]` gets discarded if Candidate 3 is eliminated, because there isn't a second-preference candidate to transfer the vote to.

In [ ]:
importlib.reload(irv);
previous_ballots = np.loadtxt('testing/previous_ballots.txt', dtype=int)
updated_ballots = np.loadtxt('testing/updated_ballots.txt', dtype=int)
candidates, vote_diffs, discarded_diff = irv.vote_transfers(previous_ballots, updated_ballots)
nbt.assert_array_equal(candidates, [1, 2, 3, 4, 5])
nbt.assert_array_equal(vote_diffs, [0, 0, -1, 0, 1])
assert discarded_diff == 0
print('Passed test.')

---

### Eliminating a candidate

Now, we know what our ballots will look like at any stage in the election. The next task is to find, during a given elimination round, which of the remaining candidates has the fewest first-preference votes and needs to be eliminated next.

You will need to write a function `eliminate_next()` which takes 1 input argument `ballots`, a NumPy array containing the current ballot data (at any stage of the election), and returns an `int` between 1 and the total number of candidates, which corresponds to the next candidate to eliminate. Your function will need to use `bd.count_votes()`.

If there are no ties, you will only need to find which of the _remaining_ candidates has the fewest first-preference votes. However, if there is a tie, i.e. if there are several candidates with the smallest number of first-preference votes, you will need to resolve the tie. To do so, compare the number of second preferences between the tied candidates. If there is still a tie with the second preferences, compare the number of third preferences, and so on, until one candidate has fewer `n`th preference votes than the other(s), then they are eliminated. In the (extremely unlikely!) event that there is still a tie after checking all preferences, you should eliminate the candidate with the smallest index. For example, if Candidates 2 and 5 are still tied, eliminate Candidate 2.

---

#### 🚩 Task 5 [4 marks]

Before writing your function, start by thinking about your testing strategy: how will you write your tests to be certain that your function eliminates the correct candidate in all possible cases, at any stage of an election, including when there are ties to resolve?

In the Markdown cell below, in no more than 300 words, explain in detail how you will test your function. For example, you could list different possible scenarios which need to be considered, and for each of these, explain how you will build `ballots` arrays with the appropriate features, and what you expect the output of your function to be.

Note that thinking about testing is **not** thinking about how you plan to write the function (keep this for Task 6!); instead, it's thinking about how you can check that your function works correctly in all possible cases. When discussing your strategy, you can assume that you will have a `bd.count_votes()` function which always gives the correct result, even if you haven't finished Task 1.

You should start with this task, and write down as full a list as you can of the scenarios and tests you will need to cover, before starting Task 6. When you do write the function later, you might come up with new ideas at that point; if you do so, come back to this task and add them here, explaining briefly how you came up with your new idea(s) in the process of coding. If you change your mind about a previous idea, explain why you changed your mind and what you will do instead. Think of it as documenting your thought process; therefore, don't worry too much about trying to write a very tidy and well-organised answer, as long as your individual ideas are each explained clearly.

For full marks, your answer should demonstrate how your tests will give sufficient evidence that your function is robust and always returns the correct result. All possible edge cases should be considered, and you should justify why your proposed tests correctly cover all the possibilities. Your answer should reflect how your ideas evolved over the course of completing this task and Task 6.

***Use this cell to write your answer to Task 5.***

---

#### 🚩 Task 6 [6 marks]

In `irv.py`, write the function `eliminate_next(ballots)` to find and return which candidate should be eliminated next.

You may obtain up to 2 marks for this question if you assume that there will be **no ties**. For full marks, your function will need to be able to resolve ties as described previously.

You will also need to add the tests you came up with in Task 5 in the code cell below.

Resolving ties correctly is a harder problem; it is strongly recommended that you start with the no-ties function, continue with Task 7, and come back to this question when you are done with Task 7.

In [ ]:
importlib.reload(irv);

# No ties: Candidate 2 should be eliminated
ballots = np.loadtxt('testing/eliminate_notie.txt', dtype=int)
assert irv.eliminate_next(ballots) == 2
print('Test passed for no-tie.')

# Tie: Candidate 3 should be eliminated
ballots = np.loadtxt('testing/eliminate_tie.txt', dtype=int)
assert irv.eliminate_next(ballots) == 3
print('Test passed for tie.')

---

### Calculating the results

Now that we have our functions to find and eliminate one candidate at a time, as well as redistribute their votes, let's run the election!

---

#### 🚩 Task 7 [4 marks]

In `irv.py`, write a function `run_election()` which takes 2 input arguments:

- `ballots`, a NumPy array containing ballot data as before,
- `display`, a Boolean with default value `False`,

and returns 2 outputs:

- `winner`, an integer between 1 and the total number of candidates, which corresponds to the candidate which has won the election,
- `eliminated`, a NumPy vector or list with the same length as the number of candidates minus 1, containing integers between 1 and the number of candidates corresponding to the eliminated candidates, in the order in which they were eliminated.

Your function should use `eliminate_next()` and `update_ballots()` to complete each elimination. If `display=True`, your function should also use `vote_transfers()` to display a summary of vote transfers at each elimination stage, in an easily readable way.

Note that there are two tests provided here, one with no ties and one with a tie to resolve. If you didn't manage to resolve ties in `eliminate_next()`, you won't be penalised for it here too; you can still get full marks for this task as long as `eliminate_next()` is used correctly in your `run_election()` function.

In [ ]:
importlib.reload(irv);
ballots = np.loadtxt('testing/election_notie.txt', dtype=int)
# Make sure to check that the transfers are displayed too!
winner, eliminated = irv.run_election(ballots, display=True)
assert winner == 1
nbt.assert_array_equal(eliminated, [3, 4, 2])
print('Passed test with no ties.')

In [ ]:
importlib.reload(irv);
ballots = np.loadtxt('testing/election_tie.txt', dtype=int)
winner, eliminated = irv.run_election(ballots, display=True)
assert winner == 4
try:
    nbt.assert_array_equal(eliminated, [1, 5, 2, 3])
    print('Passed test.')
except AssertionError:
    nbt.assert_array_equal(eliminated, [1, 2, 5, 3])
    print('Passed test with no-tie version of Task 4.')

print('Passed test with ties.')

---
### Electing multiple candidates

Finally, here is a more challenging problem. The **single transferable vote** system, or STV, can be seen as an extension to IRV, in the case where there are multiple seats to fill (instead of a single candidate to elect). For instance, Scottish council elections use this system to elect councillors. [This is a great explainer](https://ballotbox.scot/councils/stv-explained/), with an example from Glasgow from the 2017 local elections -- you can refer to this for a more detailed explanation.

The basic principle of votes transferring between candidates according to second preferences is broadly the same as in IRV. The key difference is that there are 2 possible situations where votes can be transferred:

- When a candidate is eliminated, their second preference votes are fully transferred to the other candidates (exactly as in IRV).
- When a candidate is _elected_, their **surplus** second preference votes are also transferred to the other candidates.

Instead of eliminating candidates until we are left with just one, we need to first check, at every round, whether any candidates are **elected**. A candidate is elected if they have at least a required quota of first-preference votes. This quota is calculated as:

$$
\text{quota} = \frac{\text{number of votes}}{\text{number of seats} + 1} + 1
$$

where we use integer division (floor division).

When a candidate is **elected**, their second preference votes are also transferred to other candidates, but **not as full votes**. Instead, they are **weighted** so that **only the surplus votes are redistributed**.

For example, if the quota is 100 first-preference votes to get elected, and a candidate got 125 first-preference votes, then they have 25 surplus votes to be redistributed to their second preferences. So we look at all the second preference votes on those 125 ballots, but we redistribute each of these second-preference votes as $\frac{25}{125} = 0.2$ of a vote instead. Note that this means that we end up with fractional vote counts.

When no candidates currently have enough votes to go over the quota, we go back to eliminating the least popular candidate, following the same procedure as for IRV. This time, their second-preference votes are redistributed with a weight of 1 to the other candidates (as in IRV).

---

#### 🚩 Task 8 [12 marks]

In `stv.py`, write a function `run_election_stv()` which takes 3 input arguments:

- `ballots`, a NumPy array containing ballot data as before,
- `seats`, an integer between 1 and the total number of candidates (inclusive), which corresponds to the number of seats up for election,
- `display`, a Boolean with default value `False`,

and returns 2 outputs:

- `elected`, a NumPy vector or list with length `seats` containing integers between 1 and the total number of candidates, which correspond to the elected candidates, in the order in which they were elected;
- `eliminated`, a NumPy vector or list with length given by the total number of candidates minus `seats`, containing integers between 1 and the number of candidates corresponding to the eliminated candidates, in the order in which they were eliminated.

If `display=True`, your function should also display a summary of vote transfers at each election or elimination stage, in an easily readable way.

This question is intended to be more challenging than the rest of the assignment. For full marks, you will need to demonstrate mastery of all the concepts seen in the course so far.

You will need to consider all possible situations and deal with them appropriately and efficiently, including considering the possibility that multiple candidates could be elected or eliminated in a single stage. You will also need to implement your function in an efficient way, making good use of operations with Numpy arrays where possible. Your code must be well-structured and concise, and you should write intermediate functions as appropriate to encapsulate various parts of the implementation, as you did for IRV in previous tasks.

You can re-use and/or adapt your previous functions `eliminate_next()` and `update_ballots()`; for example, you could start by adapting these to deal with electing one candidate, then think about how to generalise your implementation for situations where multiple candidates are elected in one stage (as in the first round in the linked example). Partial marks can be obtained for partial attempts (or full attempts which could be improved in various ways), therefore you can still obtain some marks even if you only manage to make a start with some of the functions you will need.

The election data [used for this example](https://ballotbox.scot/councils/stv-explained/) is provided for you in the file `testing/stv_glasgow.txt`. You can use this to test your function, and check that your results are the same as in the example.

In [ ]:
importlib.reload(stv);
ballots = np.loadtxt('testing/stv_glasgow.txt', dtype=int)
winner, eliminated = stv.run_election_stv(ballots, seats=4, display=True)